In [1]:
import sys, os, itertools
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
from gensim.corpora import Dictionary
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper
from models.bertopic import BERTopicWrapper
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = preprocess_for_bow('data.csv', preprocessing=False)

In [3]:
data.keys()

dict_keys(['text', 'ids'])

In [4]:
bertopic=BERTopicWrapper(data['text'], data['ids'])

In [5]:
bertopic.model.fit(data['text'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

(42630,)

In [6]:
len(bertopic.model.topics_) #list of topic indexes
#bertopic.model.topic_representations_
#bertopic.model.probabilities_ #probs assigned topics per documents
#bertopic.model.topic_aspects_
#bertopic.model.approximate_distribution(data['text'])[0].shape

30192

In [7]:
bertopic.model.probabilities_

array([0.53955541, 1.        , 0.        , ..., 0.99802372, 0.91082528,
       1.        ])

## TOPIC representations

In [7]:
bertopic.model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12378,-1_is_covid_the_for,"[is, covid, the, for, of, covid19, are, in, th...",[Fucking GOVERNMENT and DOCTORS and SCIENTISTS...
1,0,589,0_masks_mask_face_wearing,"[masks, mask, face, wearing, wear, ineffective...","[Can wearing masks be dangerous, Face masks ar..."
2,1,550,1_pope_church_catholic_bishops,"[pope, church, catholic, bishops, cardinal, fr...","[Not pope., Pope Francis was arrested, Does Mr..."
3,2,549,2_tweet_contribute_menu_helpful,"[tweet, contribute, menu, helpful, might, misl...",[If you want to add helpful context to any Twe...
4,3,423,3_de_en_la_un,"[de, en, la, un, el, que, del, foto, di, por]","[Este video es de un hospital en Colombia, Fot..."
...,...,...,...,...,...
672,671,6,671_tier_mental_diagnose_freud,"[tier, mental, diagnose, freud, sharkyl, sigmu...","[As a psychiatrist, I diagnose mental illness...."
673,672,6,672_smoking_1958_cigarettes_tar,"[smoking, 1958, cigarettes, tar, nonsmokers, c...",[French study published public health data whi...
674,673,6,673_newsmax_lies_repvernonjones_vickimdonovan,"[newsmax, lies, repvernonjones, vickimdonovan,...",[@RepVernonJones @ReTRUMPlican_US LIZCHENEYLIA...
675,674,6,674_cockroaches_lolor2412_petrochemical_buns,"[cockroaches, lolor2412, petrochemical, buns, ...",[Ground coffee beans contain trace elements of...


In [8]:
bertopic.model.c_tf_idf_.toarray().shape #[:,1].toarray().argmax()

(677, 42630)

## computing dot-product and cosine similarity of topic representations with respect to label -1 (most popular cluster, essentially composed of docs that do not fit well any clusters)

In [42]:
ref_vector = bertopic.model.c_tf_idf_[0].toarray().reshape(-1)
topic_to_eval=[bertopic.model.c_tf_idf_[i].toarray().reshape(-1) for i in range(len(bertopic.model.c_tf_idf_.toarray()))]
dot_products = np.dot(topic_to_eval, ref_vector)

reference_norm = np.linalg.norm(ref_vector)
vector_norms = np.linalg.norm(topic_to_eval, axis=1)
cosine_similarities = dot_products / (reference_norm * vector_norms)

In [43]:
results = [(i, dot_product, cosine_similarity) for i, (dot_product, cosine_similarity) in enumerate(zip(dot_products, cosine_similarities))]

# Sort the results by dot product or cosine similarity, depending on your preference
results_sorted_by_dot_product = sorted(results, key=lambda x: x[1], reverse=True)
results_sorted_by_cosine_similarity = sorted(results, key=lambda x: x[2], reverse=True)

### top similarity

In [44]:
results_sorted_by_cosine_similarity[:20]

[(0, 0.0005674058487948575, 1.0000000000000029),
 (21, 0.0005518146758696887, 0.4453190354097461),
 (43, 0.0005672472859508239, 0.40116779443986283),
 (6, 0.0005308921081565463, 0.39731713723304324),
 (2, 0.0005270088620527695, 0.35816610736255317),
 (39, 0.0005577420280032633, 0.356842205051339),
 (18, 0.0005155693063028026, 0.35386718101101294),
 (78, 0.0005651901286769256, 0.3466421025623777),
 (98, 0.0005687678691837359, 0.33048967437498433),
 (3, 0.000414504380397504, 0.3289540443140373),
 (59, 0.0005730422325703358, 0.32366198811850333),
 (72, 0.0005525638740014841, 0.29801359867448884),
 (11, 0.0005614887879443136, 0.29417225227524524),
 (127, 0.0005629712237382694, 0.29334556785794286),
 (10, 0.0005079279302706693, 0.29278665421496136),
 (38, 0.0005488873155859692, 0.2856231281949281),
 (125, 0.0005710188929900809, 0.279588988258647),
 (106, 0.0005654037516093839, 0.27667486536599295),
 (33, 0.0005423249441409185, 0.2724246768886796),
 (51, 0.0005549981529086999, 0.270075848272

In [45]:
bertopic.model.get_topic_info().iloc[[4,23,3,41,1,128,7,56,16,169,26,8,108]]

,Topic,Count,Name,Representation,Representative_Docs
4,3,356,3_de_en_la_un,"[de, en, la, un, del, el, foto, que, di, da]",[Ya saltó un p^%*~o a decir que “hay que aboli...
23,22,98,22_soldiers_galwan_valley_chinese,"[soldiers, galwan, valley, chinese, indian, ar...","[30 Chinese soldiers killed in Galwan Valley, ..."
3,2,474,2_pope_church_catholic_bishops,"[pope, church, catholic, bishops, cardinal, fr...",[As for the new things which we can find in th...
41,40,71,40_vaccines_vaccine_efficacy_covid19,"[vaccines, vaccine, efficacy, covid19, effecti...",[BREAKING—Novavax says its #COVID19 vaccine is...
1,0,550,0_tweet_contribute_menu_helpful,"[tweet, contribute, menu, helpful, might, misl...",[If you want to add helpful context to any Twe...
128,127,30,127_je_bu_pandemija_da,"[je, bu, pandemija, da, toont, meso, dokazuje,...",[Video iz bolnice u Izraelu je dokaz da je pan...
7,6,266,6_nigeria_nigerian_nigerias_nigerians,"[nigeria, nigerian, nigerias, nigerians, ghana...","[Nigeria has discovered a Covid-19 vaccine, Im..."
56,55,55,55_facebook_facebooks_users_posts,"[facebook, facebooks, users, posts, page, lord...",[Facebook has adopted a new privacy policy giv...
16,15,128,15_5g_radiation_towers_cells,"[5g, radiation, towers, cells, technology, ele...","[5G can make you sick., Coronavirus is a resul..."
169,168,26,168_population_sms_urge_uniform,"[population, sms, urge, uniform, corona, crisi...",[Once our country comes out of this present cr...


### bottom similarity docs

In [46]:
results_sorted_by_cosine_similarity[-20:]

[(636, 0.0004887599291525421, 0.033222276861133125),
 (622, 0.00047768181752810054, 0.033059863353496956),
 (386, 0.00041829825136523245, 0.03288810297067284),
 (316, 0.00031587659763883673, 0.03277039070515132),
 (607, 0.0004179196499600695, 0.032049501601586566),
 (448, 0.00042503595841760606, 0.031848295094590556),
 (223, 0.0002884481610444074, 0.03170610462131625),
 (524, 0.0003350140280592074, 0.03107992979795166),
 (319, 0.0002893859792129378, 0.030017773628538844),
 (534, 0.0003900563944254968, 0.02992110265295896),
 (468, 0.00040443168855681456, 0.02989893950216143),
 (423, 0.0003988114170784051, 0.027282943528572128),
 (662, 0.00046731520480269476, 0.02536764793720587),
 (623, 0.00037107627746883596, 0.02327328884301325),
 (495, 0.00030761217464105934, 0.02080131180387816),
 (353, 0.00021482166163361313, 0.020715461270562764),
 (617, 0.00030847522674244557, 0.019377441820659407),
 (128, 0.00016510399619345953, 0.019258040859207556),
 (580, 0.0002568435436492968, 0.017895095624

In [47]:
bertopic.model.get_topic_info().iloc[[398, 534, 363, 585, 643, 411]]

,Topic,Count,Name,Representation,Representative_Docs
398,397,12,397_kissing_young_cheek_girl,"[kissing, young, cheek, girl, inappropriately,...",[A photograph captures Joe Biden posing inappr...
534,533,9,533_shakespeare_caesar_julius_govern,"[shakespeare, caesar, julius, govern, idiots, ...","[""""What a Terrible Era in Which Idiots Govern ..."
363,362,13,362_pipeline_keystone_oil_spill,"[pipeline, keystone, oil, spill, earthjustice,...",[From cancelling the Keystone Pipeline to bann...
585,584,8,584_costco_rotisserie_crispy_499,"[costco, rotisserie, crispy, 499, chaokoh, cos...",[Costco closed stores across the U.S. in Febru...
643,642,7,642_succeeding_antithesis_politic_fulfil,"[succeeding, antithesis, politic, fulfil, nesa...",[from the Left. (How can you blame him?) They ...
411,410,12,410_fruits_tomato_tomatoes_affecting,"[fruits, tomato, tomatoes, affecting, transmit...",[Coronavirus can spread through vegetables and...


In [48]:
results_sorted_by_cosine_similarity[100:120]

[(420, 0.0006037834302813089, 0.14928026158029076),
 (366, 0.0005497637933415418, 0.14922776427252604),
 (32, 0.00045507190233883867, 0.14806826003117302),
 (75, 0.0005077068357828437, 0.1476314752517955),
 (52, 0.0005358536115585577, 0.14762481619692758),
 (155, 0.0005294229555590775, 0.14761124285678456),
 (154, 0.0005651132248324375, 0.14753022822684475),
 (240, 0.0005322070002938852, 0.14659887589868154),
 (55, 0.000582367781351782, 0.1465062100927746),
 (571, 0.0005850859336649043, 0.14635189417967118),
 (543, 0.0006030206110332615, 0.1459921896716473),
 (263, 0.0005503496423277156, 0.1423050551096984),
 (295, 0.0005220420942666742, 0.1418901639535388),
 (336, 0.0005674454327182059, 0.14166785982433414),
 (123, 0.0005784517599418278, 0.14156627578160785),
 (13, 0.00041754531575200767, 0.13978926423221125),
 (140, 0.0005122692774953307, 0.13737036593126875),
 (222, 0.0005582669770067346, 0.1373550062440667),
 (63, 0.0005389887621946288, 0.13730099630415965),
 (190, 0.00057187021436

In [49]:
bertopic.model.get_topic_info().iloc[[35, 460, 229, 214, 151]]

,Topic,Count,Name,Representation,Representative_Docs
35,34,76,34_makhia_rittenhouse_kenosha_kyle,"[makhia, rittenhouse, kenosha, kyle, bryant, f...","[A video shows Kyle Rittenhouse, the 17-year-o..."
460,459,11,459_0426_risk_earthquake_high,"[0426, risk, earthquake, high, angeles, 0427, ...",[Strong earthquake is likely in the Los Angele...
229,228,21,228_corona_patient_gvt_alertstay,"[corona, patient, gvt, alertstay, barhalganj, ...","[Corona Patient In Kanpur, Uttar Pradesh. Be A..."
214,213,22,213_thailand_thai_thailands_curfew,"[thailand, thai, thailands, curfew, nationwide...",[Thailand health minister had panic attack bef...
151,150,27,150_ethiopia_ethiopian_tplf_tigray,"[ethiopia, ethiopian, tplf, tigray, ethiopias,...","[⭕️ #Ethiopia .. #TigrayGenocide, Photo shows ..."


In [50]:
results_sorted_by_cosine_similarity[45:65]

[(34, 0.0005221001569764934, 0.19585491696445487),
 (49, 0.0005165594979936125, 0.19543474332485275),
 (306, 0.0005690731315094034, 0.1936205530686779),
 (20, 0.0005407142571144918, 0.19197966382798323),
 (86, 0.0005025952005753079, 0.19094056197950654),
 (35, 0.000523602441859423, 0.19073089696897289),
 (7, 0.0004765988026894797, 0.18955771743679173),
 (9, 0.0004530062182971779, 0.18744424162559742),
 (174, 0.0005745864352708241, 0.18404034802148234),
 (207, 0.0005484504683976605, 0.183857371925276),
 (477, 0.0005707555129270995, 0.18117627409179907),
 (88, 0.0005219197588938174, 0.18070579699323414),
 (25, 0.0005513175489945332, 0.17978198510967633),
 (205, 0.0005982539951272646, 0.17917833291309054),
 (57, 0.0005655431079921478, 0.17915530008335126),
 (31, 0.0005449324352512223, 0.17827082269622832),
 (169, 0.0005657778036252939, 0.17822566917795962),
 (287, 0.0005398242991548071, 0.1770678163073252),
 (104, 0.0005514599913173698, 0.1750433348968662),
 (490, 0.0005582256254170542, 0

In [51]:
bertopic.model.get_topic_info().iloc[[89, 315, 48, 86, 173, 19, 47]]

,Topic,Count,Name,Representation,Representative_Docs
89,88,38,88_daunte_wright_freshener_air,"[daunte, wright, freshener, air, mirror, daunt...","[On Sunday afternoon in Minnesota, Daunte Wrig..."
315,314,15,314_tears_net_worth_connery,"[tears, net, worth, connery, sean, left, famil...",[Pierce Brosnan's final net worth left his fam...
48,47,62,47_iran_realdonaldtrump_iranian_irans,"[iran, realdonaldtrump, iranian, irans, nuclea...",[I predict that President Obama will at some p...
86,85,39,85_fbi_manafort_investigation_comey,"[fbi, manafort, investigation, comey, strzok, ...",[The report identified five officials with som...
173,172,25,172_tucker_carlson_carlsons_nsa,"[tucker, carlson, carlsons, nsa, milley, fox, ...",[Tucker Carlson reveals the Biden administrati...
19,18,107,18_water_drinking_cure_salt,"[water, drinking, cure, salt, lemon, tea, ging...",[Drinking garlic water can cure novel coronavi...
47,46,64,46_bjp_mla_bihar_anil,"[bjp, mla, bihar, anil, caa, seats, nrc, mlas,...",[BJP MLA Anil Upadhayay criticising CAA and NR...


In [52]:
results_sorted_by_cosine_similarity[25:40]

[(53, 0.0005395959701205074, 0.24082763279292652),
 (41, 0.0005444749902392469, 0.2397395408874364),
 (64, 0.000546996341896771, 0.22328685597195488),
 (16, 0.0005135870009932596, 0.22275610031648685),
 (37, 0.0005694214208278598, 0.21865637053762652),
 (12, 0.0005568125291171581, 0.2165671080728374),
 (65, 0.0005573132212961804, 0.21622920933996323),
 (198, 0.0005626764705536445, 0.2161610415695773),
 (77, 0.0005395650040472, 0.21533069157784815),
 (598, 0.0006045928641925163, 0.21204059060279293),
 (177, 0.0005267250073949126, 0.20924778633678914),
 (241, 0.0005630594837564527, 0.20671461754789694),
 (283, 0.0005534028313179201, 0.2029241783729289),
 (291, 0.0005904081249494394, 0.19978283109025544),
 (48, 0.0005210632501388773, 0.19881361992985166)]

In [53]:
bertopic.model.get_topic_info().iloc[[27, 58, 13, 77, 62, 65]]

,Topic,Count,Name,Representation,Representative_Docs
27,26,84,26_won_113_rightfully_trump,"[won, 113, rightfully, trump, patriots, landsl...","[@POTUS TRUMP WON, @Leverich1991 @Girth_Daddy ..."
58,57,54,57_delhi_stones_riots_tahir,"[delhi, stones, riots, tahir, pelting, gaffar,...",[Video of violent anti-CAA protesters pelting ...
13,12,168,12_mattythrice_rickastley_troll_aoc,"[mattythrice, rickastley, troll, aoc, joebiden...",[@AsTheWorldBurnz @MattyThrice @caeser_pounce ...
77,76,45,76_aoc_she_alexandriaocasiosmollett_her,"[aoc, she, alexandriaocasiosmollett, her, buil...",[AOC Wasn't Even in the Capitol Building Durin...
62,61,51,61_marjorie_greene_taylor_rep,"[marjorie, greene, taylor, rep, congresswoman,...",[US Congresswoman Marjorie Taylor Greene said:...
65,64,50,64_deaths_toll_cdc_death,"[deaths, toll, cdc, death, 94, centers, mortal...","[The CDC """"quietly updated"""" its COVID-19 mort..."


In [54]:
bertopic.model.get_topic_info().iloc[[99, 34, 30, 12, 6, 37, 138]]

,Topic,Count,Name,Representation,Representative_Docs
99,98,35,98_electric_cars_tesla_autopilot,"[electric, cars, tesla, autopilot, car, vehicl...","[In June 2021, California officials told resid..."
34,33,76,33_climate_emissions_warming_change,"[climate, emissions, warming, change, renewabl...",[Climate change isn’t a national security issu...
30,29,82,29_sri_lanka_lankan_lankas,"[sri, lanka, lankan, lankas, ranked, gotabaya,...",[Picture shows real banner at an opposition pa...
12,11,174,11_woof_am_fuck_sums,"[woof, am, fuck, sums, wild, yo, what, boi, th...","[I am become meme, Destroyer of shorts, 🚨 New ..."
6,5,303,5_wuhan_china_lab_bioweapon,"[wuhan, china, lab, bioweapon, virus, bioweapo...",[COVID came from the Wuhan Lab of Virology in ...
37,36,75,36_fauci_anthony_dr_faucis,"[fauci, anthony, dr, faucis, niaid, firefauci,...",[Dr Anthony Fauci has been fired for the US' h...
138,137,29,137_stairs_allegiance_joe_pledge,"[stairs, allegiance, joe, pledge, biden, aslee...",[Joe Biden botched recitation of the Pledge of...


## THRESHOLD of 0.25/0.24 seems appropriate 

# RESPECT TO label -1

In [55]:
topic_keep=[i-1 for i in np.where(cosine_similarities>=0.25)[0]]
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i] in topic_keep]
newtext = [data['text'][i] for i in idx_to_keep]
newids = [data['ids'][i] for i in idx_to_keep]

In [56]:
bertopic2=BERTopicWrapper(newtext, newids)
bertopic2.model.fit(newtext)

In [57]:
bertopic2.model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6746,-1_the_of_to_in,"[the, of, to, in, is, and, for, covid19, that,...",[You know you can’t catch a fake virus right.....
1,0,606,0_masks_mask_face_wearing,"[masks, mask, face, wearing, wear, not, you, a...","[Can wearing masks be dangerous, Stop wearing ..."
2,1,574,1_pope_church_catholic_bishops,"[pope, church, catholic, bishops, francis, car...","[And at a later date, he assured us that that ..."
3,2,475,2_china_wuhan_chinese_lab,"[china, wuhan, chinese, lab, bioweapon, virus,...","[President Barack Obama, Dr Anthony Fauci and ..."
4,3,395,3_biden_joe_bidens_president,"[biden, joe, bidens, president, presidential, ...",[Claims made at the presidential debate by Don...
...,...,...,...,...,...
218,217,6,217_nasal_pneumonia_blends_fluid,"[nasal, pneumonia, blends, fluid, trachea, las...",[Covid-19 starts with a sore throat lasting 3-...
219,218,6,218_driver_rohingya_engineer_pakistani,"[driver, rohingya, engineer, pakistani, beats,...",[A recent video shows a Chinese engineer caugh...
220,219,6,219_olive_pasta_garden_hannitys,"[olive, pasta, garden, hannitys, revoke, pass,...","[Olive Garden is supporting Trump’s bigotry, c..."
221,220,6,220_infant_mother_riding_train,"[infant, mother, riding, train, carrying, aata...",[Mother and infant traveling in train dangerou...


In [58]:
idx_minus_one = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i]==-1]
[data['text'][i] for i in idx_minus_one]

['I can even go so far to say that I think the thing (mood) boys have when they think they over-smarted the media and cured a pandemic is cute and I truly feel like a b*tch to poke on that. I thought about it a lot not say anything to avoid make people angry would be selfish.',
 'Election Fraud Covid19 The V.a.c.c.i.n.e. Satanic Pedophilia Masks Social Distancing New World Order BLM ANTIFA White Privilege Racism Capitol Insurrection Cities On Fire Gun Control Assault Rifle Ban Destroy the US Constitution Slavery Reparations The List Goes On... #TheBigLie',
 'Day 48. No microchips in vaccines. COVID didn’t suddenly disappear on Nov 4 like the GOP predicted. No socialism deadly migrant caravans drive thru abortion clinics ANTIFA MS13. But this new guy is great. He’s done way more than golfing guy did his entire 4 years in office.',
 'Hello can one report a channel that deals essentially in misleading information (e.g. about COVID-19 masks etc.) and in conspiracy theories (Qanon-style stu

In [61]:
idx_minus_one = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i]==1]
[data['text'][i] for i in idx_minus_one]

['....ELDERLY ... IT MAY PROTECT YOU SOME....OTHERWISE THE ONLY USE FOR THE MASK IS TO MIND CONTROL YOU WITH FEAR GENERATION SO THEY CAN STRIP YOU OF YOUR RIGHTS. COVID19 DID NOT AND IS NOT GOING TO KILL BY THE MASSES. THROW THE FACE MASK AWAY',
 'I haven’t seen any arrest for No face masks and no social distancing....so....this false flag has just broken the death grip you have on us law abiding citizens!!! Your comrades screwed you bad. 🤣 Riots have CURED YOUR COVID HOAX YOU MURDERER',
 'All on this thread.... sheeple so much for science about effectiveness of masks some studies say they work others show they’re useless even if everyone at the rally got Covid at .99% survival rate maybe 10 out of 1000 die maybe. 10/1000 will die in auto wrecks.',
 "Yeah. Right. Your defiance to masks and mask orders thus ensuring that this pandemic continues can only come from one of three places: the President the hack right-wing media that supports him or QANON and their ilk. If you actually did yo

In [ ]:
idx_minus_one = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i]==2]
[data['text'][i] for i in idx_minus_one]

# RESPECT TO LABEL 0

In [34]:
ref_vector = bertopic.model.c_tf_idf_[1].toarray().reshape(-1)
topic_to_eval=[bertopic.model.c_tf_idf_[i].toarray().reshape(-1) for i in range(len(bertopic.model.c_tf_idf_.toarray()))]
dot_products = np.dot(topic_to_eval, ref_vector)

In [35]:
reference_norm = np.linalg.norm(ref_vector)
vector_norms = np.linalg.norm(topic_to_eval, axis=1)
cosine_similarities = dot_products / (reference_norm * vector_norms)

In [36]:
results = [(i, dot_product, cosine_similarity) for i, (dot_product, cosine_similarity) in enumerate(zip(dot_products, cosine_similarities))]

results_sorted_by_dot_product = sorted(results, key=lambda x: x[1], reverse=True)
results_sorted_by_cosine_similarity = sorted(results, key=lambda x: x[2], reverse=True)

In [37]:
results_sorted_by_cosine_similarity[:20]

[(1, 0.013874230536754788, 1.0),
 (26, 0.0038313035722291767, 0.1701086910008799),
 (46, 0.003158490633629975, 0.15430529977347304),
 (533, 0.00232745470644959, 0.07835837669322643),
 (0, 0.00014038449749733572, 0.05003429610876782),
 (12, 0.00048715495604095203, 0.03831713898082409),
 (21, 0.00022297819127159945, 0.03639006674424163),
 (439, 0.0010584128867750337, 0.03390778597814353),
 (43, 0.0002146116457285562, 0.03069370889583635),
 (362, 0.00045863439661158795, 0.028795561012362532),
 (6, 0.00017914436215322766, 0.02711294068637876),
 (72, 0.00023829487873555037, 0.025990267757864716),
 (462, 0.000622183099342544, 0.025264304411390555),
 (571, 0.0004889747146877186, 0.02473475103296486),
 (3, 0.00015395390854744428, 0.024708095752109072),
 (33, 0.00024319048831559993, 0.024704490405843677),
 (39, 0.0001881787223358558, 0.02434758043239815),
 (78, 0.00019510143882252195, 0.024198567929571813),
 (18, 0.00017322744509176306, 0.024044313560486447),
 (59, 0.0001929117811245235, 0.0220

In [40]:
bertopic.model.get_topic_info().iloc[[26, 46, 533, 0, 12, 21, 439, 43, 6, 3, 78, 59]]

,Topic,Count,Name,Representation,Representative_Docs
26,25,84,25_valuable_identify_guide_jump,"[valuable, identify, guide, jump, youd, writin...",[@botz_tv Welcome to Birdwatch! 🎉 You can now ...
46,45,65,45_howto_notes_site_check,"[howto, notes, site, check, guide, jump, youd,...",[@_jake_holland Welcome to Birdwatch! 🎉 You ca...
533,532,9,532_notes_rating_elevate_view,"[notes, rating, elevate, view, site, rate, hel...",[You can also help by rating notes — it will e...
0,-1,12197,-1_covid_of_is_the,"[covid, of, is, the, for, are, in, with, covid...",[Its crazy when its at the convenience of the ...
12,11,174,11_woof_am_fuck_sums,"[woof, am, fuck, sums, wild, yo, what, boi, th...","[I am become meme, Destroyer of shorts, 🚨 New ..."
21,20,106,20_pandemic_plandemic_end_shit,"[pandemic, plandemic, end, shit, over, control...",[This isn’t a pandemic! It’s a plandemic and w...
439,438,11,438_feedback_uptodate_dm_pilot,"[feedback, uptodate, dm, pilot, directly, take...",[We can't wait to see where this pilot takes u...
43,42,71,42_microchip_microchips_vaccine_covid,"[microchip, microchips, vaccine, covid, dont, ...",[Also what do they think these microchips do e...
6,5,303,5_wuhan_china_lab_bioweapon,"[wuhan, china, lab, bioweapon, virus, bioweapo...",[COVID came from the Wuhan Lab of Virology in ...
3,2,474,2_pope_church_catholic_bishops,"[pope, church, catholic, bishops, cardinal, fr...",[As for the new things which we can find in th...


In [33]:
results_sorted_by_cosine_similarity[20:50]

[(24, 0.0005291033062557692, 0.25775480723557864),
 (61, 0.0005406788130561397, 0.2563766147622202),
 (17, 0.000516015753453191, 0.2526775870694589),
 (235, 0.000565217904556622, 0.24323232668391315),
 (5, 0.0004932838119761068, 0.24152511308203595),
 (53, 0.0005395959701205074, 0.24082763279292652),
 (41, 0.0005444749902392469, 0.2397395408874364),
 (64, 0.000546996341896771, 0.22328685597195488),
 (16, 0.0005135870009932596, 0.22275610031648685),
 (37, 0.0005694214208278598, 0.21865637053762652),
 (12, 0.0005568125291171581, 0.2165671080728374),
 (65, 0.0005573132212961804, 0.21622920933996323),
 (198, 0.0005626764705536445, 0.2161610415695773),
 (77, 0.0005395650040472, 0.21533069157784815),
 (598, 0.0006045928641925163, 0.21204059060279293),
 (177, 0.0005267250073949126, 0.20924778633678914),
 (241, 0.0005630594837564527, 0.20671461754789694),
 (283, 0.0005534028313179201, 0.2029241783729289),
 (291, 0.0005904081249494394, 0.19978283109025544),
 (48, 0.0005210632501388773, 0.198813

In [41]:
bertopic.model.get_topic_info().iloc[[5, 41, 16, 12, 65, 177, 45, 130, 56]]

,Topic,Count,Name,Representation,Representative_Docs
5,4,323,4_ballots_mailin_ballot_votes,"[ballots, mailin, ballot, votes, voter, fraud,...",[More ballots were cast in Wisconsin than the ...
41,40,71,40_vaccines_vaccine_efficacy_covid19,"[vaccines, vaccine, efficacy, covid19, effecti...",[BREAKING—Novavax says its #COVID19 vaccine is...
16,15,128,15_5g_radiation_towers_cells,"[5g, radiation, towers, cells, technology, ele...","[5G can make you sick., Coronavirus is a resul..."
12,11,174,11_woof_am_fuck_sums,"[woof, am, fuck, sums, wild, yo, what, boi, th...","[I am become meme, Destroyer of shorts, 🚨 New ..."
65,64,50,64_deaths_toll_cdc_death,"[deaths, toll, cdc, death, 94, centers, mortal...","[The CDC """"quietly updated"""" its COVID-19 mort..."
177,176,25,176_beast_mark_coming_revelation,"[beast, mark, coming, revelation, antichrist, ...",[Remember: Revelation 13 is clear on “The Mark...
45,44,67,44_abortion_babies_parenthood_abortions,"[abortion, babies, parenthood, abortions, birt...",[Our taxpayer dollars should NEVER go to fundi...
130,129,30,129_racism_racist_white_rutgers,"[racism, racist, white, rutgers, license, tire...","[In June 2020, Merriam-Webster said it planned..."
56,55,55,55_facebook_facebooks_users_posts,"[facebook, facebooks, users, posts, page, lord...",[Facebook has adopted a new privacy policy giv...


In [121]:
topic_keep=[i-1 for i in np.where(cosine_similarities>=0.25)[0]]
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i] in topic_keep]
newtext = [data['text'][i] for i in idx_to_keep]
newids = [data['ids'][i] for i in idx_to_keep]

In [122]:
bertopic3=BERTopicWrapper(newtext, newids)
bertopic3.model.fit(newtext)

In [ ]:
bertopic3.model.get_topic_info()

# FIXING with fixed set of topics

In [4]:
bertopic=BERTopicWrapper(data['text'], data['ids'], nb_topics=9)
bertopic.model.fit(data['text'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [5]:
bertopic.model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12415,-1_the_of_to_in,"[the, of, to, in, and, is, for, that, on, are]",[Haha you act like Covid is his fault! He save...
1,0,6168,0_the_to_of_is,"[the, to, of, is, and, covid19, in, for, that,...",[A big deal is that the % of deaths attributed...
2,1,5526,1_the_to_in_of,"[the, to, in, of, and, trump, is, that, for, p...","[Just to be clear, President Trump won the 202..."
3,2,4952,2_in_the_of_shows,"[in, the, of, shows, to, and, video, is, on, for]",[This video shows an explosion in Wuhan in lat...
4,3,729,3_tweet_to_birdwatch_context,"[tweet, to, birdwatch, context, add, try, misl...",[If you want to add helpful context to any Twe...
5,4,223,4_schools_safe_school_are,"[schools, safe, school, are, to, and, the, in,...","[Reminder schools are safe., @vaidehibajpai Sc..."
6,5,157,5_cancer_oxygen_can_and,"[cancer, oxygen, can, and, water, of, drinking...",[f the oxygen level is going down and there is...
7,6,15,6_slaves_slavery_the_were,"[slaves, slavery, the, were, white, to, coloni...",[“Slavery in this land was not merely an unfor...
8,7,7,7_robot_bodyguard_bahrain_dubai,"[robot, bodyguard, bahrain, dubai, king, arriv...",[King of Bahrain arrives in Dubai with his rob...


In [51]:
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 2), min_df=10) #can override preprocessor
                                            #strip_accents
                                            #min_df (min occurence over the whole corpus ?)
bertopic.model.update_topics(data['text'], vectorizer_model=vectorizer_model)

In [52]:
bertopic.model.c_tf_idf_.toarray().shape

(658, 3096)

In [17]:
#bertopic.model.get_document_info(data['text'])

In [18]:
#bertopic.model.get_representative_docs()

In [19]:
#bertopic.model.generate_topic_labels()

In [ ]:
bertopic.model.merge_topics(docs, topics_to_merge)

In [ ]:
bertopic.model.reduce_outliers(docs, topics)